<a href="https://colab.research.google.com/github/MPdeM/Final_Project/blob/master/machinelearning/nlp_svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install nltk

### **Dependencies**

In [2]:
# Dependencies
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk import word_tokenize,sent_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, svm
from sklearn.metrics import accuracy_score
from time import time

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jobrien1726/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jobrien1726/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jobrien1726/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jobrien1726/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## **Load Data**

In [3]:
# Read in csv file
url = "https://job-postings-dataviz.s3.amazonaws.com/fake_jobs_clean.csv"

fake_jobs_df = pd.read_csv(url, sep=",",  encoding = "UTF-8")
fake_jobs_df.head()

,job_id,city,state/province,country,title,department,industry,function,salary_range,salary_provided,...,requirements,benefits,benefits_provided,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,fraudulent
0,1,New York,NY,US,Marketing Intern,Marketing,NaN,Marketing,NaN,0,...,Experience with content management systems a m...,NaN,0,0,1,0,Other,Internship,NaN,0
1,2,Auckland,,NZ,Customer Service - Cloud Video Production,Success,Marketing and Advertising,Customer Service,NaN,0,...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,1,0,1,0,Full-time,Not Applicable,NaN,0
2,3,Wever,IA,US,Commissioning Machinery Assistant (CMA),NaN,NaN,NaN,NaN,0,...,Implement pre-commissioning and commissioning ...,NaN,0,0,1,0,NaN,NaN,NaN,0
3,4,Washington,DC,US,Account Executive - Washington DC,Sales,Computer Software,Sales,NaN,0,...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,1,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,0
4,5,Fort Worth,FL,US,Bill Review Manager,NaN,Hospital & Health Care,Health Care Provider,NaN,0,...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,1,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,0


## **Data Cleaning**

In [13]:
# Take a look at job description and fraudulent cols
df = fake_jobs_df[['description', 'fraudulent']]
df

,description,fraudulent
0,"Food52, a fast-growing, James Beard Award-winn...",0
1,Organised - Focused - Vibrant - Awesome!Do you...,0
2,"Our client, located in Houston, is actively se...",0
3,THE COMPANY: ESRI – Environmental Systems Rese...,0
4,JOB TITLE: Itemization Review ManagerLOCATION:...,0
...,...,...
17875,Just in case this is the first time you’ve vis...,0
17876,The Payroll Accountant will focus primarily on...,0
17877,Experienced Project Cost Control Staff Enginee...,0
17878,Nemsia Studios is looking for an experienced v...,0


In [14]:
# Count how many Real vs Fraudulent postings 
from collections import Counter
print(Counter(df['fraudulent'].values))

Counter({0: 17014, 1: 866})


In [15]:
# Drop duplicate descriptions
df = df.drop_duplicates()
df

,description,fraudulent
0,"Food52, a fast-growing, James Beard Award-winn...",0
1,Organised - Focused - Vibrant - Awesome!Do you...,0
2,"Our client, located in Houston, is actively se...",0
3,THE COMPANY: ESRI – Environmental Systems Rese...,0
4,JOB TITLE: Itemization Review ManagerLOCATION:...,0
...,...,...
17873,RESPONSIBILITIES:Will facilitate the recruitin...,0
17875,Just in case this is the first time you’ve vis...,0
17876,The Payroll Accountant will focus primarily on...,0
17877,Experienced Project Cost Control Staff Enginee...,0


In [16]:
# Count how many Real vs Fraudulent postings 
print(Counter(df['fraudulent'].values))

Counter({0: 14171, 1: 632})


In [17]:
# Make sure number of real vs fraudulent are more balanced
# Using UNDERsampling for balancing of classification groups
df_fraud= df[df['fraudulent'] == 1] 
df_normal = df[df['fraudulent'] == 0] 

df_normal = df_normal.sample(n=(4*len(df_fraud)), random_state=22)
df = df_normal.append(df_fraud)

df

,description,fraudulent
13898,Currently hiring per-diem caregivers to work o...,0
1688,"Μεγαλώνουμε την ομάδα μας,και χρειαζόμαστε ένα...",0
13100,PowToon is looking for a team-oriented Marketi...,0
5579,Wealth Management Advisor We are continuing ou...,0
15961,RECRUITING MILITARY VETERANSLooking to offer o...,0
...,...,...
17827,Student Positions Part-Time and Full-Time.You ...,1
17828,LEARN TO EARN AN EXECUTIVE LEVEL INCOMEFULL TR...,1
17829,inFullMobile Sp. z o.o. is a mobile software d...,1
17830,JOB DESCRIPTIONWe are seeking a full time payr...,1


In [18]:
# Shuffle the dataframe so fraudulent postings more evenly distributed 
from sklearn.utils import shuffle
df = shuffle(df, random_state=22)
df = df.reset_index(drop=True)
df

,description,fraudulent
0,"About the roleAre you drawn to simple, beautif...",0
1,"Hi! We are Netguru! We design, create, impleme...",0
2,We're looking for a Test Analyst to join our A...,0
3,PURPOSE OF JOB: The Grants Manager plans and...,0
4,Sewing machinist needed by friendly London-bas...,0
...,...,...
3155,CTO for a tourism start-upWe are just now star...,0
3156,Job Description: Candidate should have a stron...,1
3157,The teamBackbase is an established and Custome...,0
3158,As a Web Engineer at Runscope you'll be respon...,0


In [19]:
# Explicitly define data types
df['description'] = df['description'].astype(str)
df['fraudulent'] = df['fraudulent'].astype(int)

In [20]:
# Datatypes
df.dtypes

description    object
fraudulent      int64
dtype: object

In [21]:
# Count Real vs Fraudulent again to make sure we've achieved the desired result
print(Counter(df['fraudulent'].values))

Counter({0: 2528, 1: 632})


## **Pre-Processing**

In [22]:
# Change all the text to lowercase
df['description'] = [x.lower() for x in df['description']]
df['description']

0       about the roleare you drawn to simple, beautif...
1       hi! we are netguru! we design, create, impleme...
2       we're looking for a test analyst to join our a...
3       purpose of job:   the grants manager plans and...
4       sewing machinist needed by friendly london-bas...
                              ...                        
3155    cto for a tourism start-upwe are just now star...
3156    job description: candidate should have a stron...
3157    the teambackbase is an established and custome...
3158    as a web engineer at runscope you'll be respon...
3159    nursing &amp; therapy positions-home care&nbsp...
Name: description, Length: 3160, dtype: object

In [23]:
# Tokenization
df['description']= [word_tokenize(x) for x in df['description']]

In [24]:
# Remove Stop words, Perfom Word Stemming/Lemmatization
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

for index,entry in enumerate(df['description']):
    # Declare an Empty List to store the words that follow the rules for this step
    Final_words = []

    # Initialize WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()

    # pos_tag function to provide the 'tag' i.e if the word is Noun(N), Verb(V), ADJ(J) or ADV(R) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)

    # The final processed set of words for each iteration will be stored in 'text_final'
    df.loc[index,'text_final'] = str(Final_words)

df

,description,fraudulent,text_final
0,"[about, the, roleare, you, drawn, to, simple, ...",0,"['roleare', 'draw', 'simple', 'beautiful', 'de..."
1,"[hi, !, we, are, netguru, !, we, design, ,, cr...",0,"['hi', 'netguru', 'design', 'create', 'impleme..."
2,"[we, 're, looking, for, a, test, analyst, to, ...",0,"['look', 'test', 'analyst', 'join', 'auckland'..."
3,"[purpose, of, job, :, the, grants, manager, pl...",0,"['purpose', 'job', 'grant', 'manager', 'plan',..."
4,"[sewing, machinist, needed, by, friendly, lond...",0,"['sew', 'machinist', 'need', 'friendly', 'toy'..."
...,...,...,...
3155,"[cto, for, a, tourism, start-upwe, are, just, ...",0,"['cto', 'tourism', 'start', 'want', 'join', 'a..."
3156,"[job, description, :, candidate, should, have,...",1,"['job', 'description', 'candidate', 'strong', ..."
3157,"[the, teambackbase, is, an, established, and, ...",0,"['teambackbase', 'establish', 'customer', 'exp..."
3158,"[as, a, web, engineer, at, runscope, you, 'll,...",0,"['web', 'engineer', 'runscope', 'responsible',..."


In [25]:
# Reorganize Columns
df = df[['description','text_final','fraudulent']]
df

,description,text_final,fraudulent
0,"[about, the, roleare, you, drawn, to, simple, ...","['roleare', 'draw', 'simple', 'beautiful', 'de...",0
1,"[hi, !, we, are, netguru, !, we, design, ,, cr...","['hi', 'netguru', 'design', 'create', 'impleme...",0
2,"[we, 're, looking, for, a, test, analyst, to, ...","['look', 'test', 'analyst', 'join', 'auckland'...",0
3,"[purpose, of, job, :, the, grants, manager, pl...","['purpose', 'job', 'grant', 'manager', 'plan',...",0
4,"[sewing, machinist, needed, by, friendly, lond...","['sew', 'machinist', 'need', 'friendly', 'toy'...",0
...,...,...,...
3155,"[cto, for, a, tourism, start-upwe, are, just, ...","['cto', 'tourism', 'start', 'want', 'join', 'a...",0
3156,"[job, description, :, candidate, should, have,...","['job', 'description', 'candidate', 'strong', ...",1
3157,"[the, teambackbase, is, an, established, and, ...","['teambackbase', 'establish', 'customer', 'exp...",0
3158,"[as, a, web, engineer, at, runscope, you, 'll,...","['web', 'engineer', 'runscope', 'responsible',...",0


## **Training/Testing Data**

In [26]:
# Prepare Train and Test Data sets
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(df['text_final'],df['fraudulent'],test_size=0.3)

## **Word Vectorization w TF-IDF**

In [27]:
# Word Vectorization using method TF-IDF
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(df['text_final'])
X_train_Tfidf = Tfidf_vect.transform(X_train)
X_test_Tfidf = Tfidf_vect.transform(X_test)

# View the vocab its learned
print(Tfidf_vect.vocabulary_)
print(X_train_Tfidf)

{'draw': 1365, 'simple': 4100, 'beautiful': 456, 'design': 1223, 'want': 4851, 'help': 2089, 'shape': 4057, 'brand': 553, 'people': 3234, 'around': 293, 'world': 4943, 'come': 819, 'know': 2492, 'love': 2626, 'designer': 1227, 'marketing': 2706, 'team': 4435, 'chance': 694, 'establish': 1584, 'intercom': 2361, 'create': 1059, 'material': 2715, 'inspire': 2317, 'customer': 1113, 'change': 695, 'face': 1702, 'service': 4035, 'mission': 2825, 'make': 2666, 'web': 4868, 'mobile': 2832, 'business': 602, 'personal': 3260, 'provide': 3535, 'messaging': 2780, 'easy': 1409, 'way': 4863, 'company': 849, 'engage': 1510, 'meaningful': 2735, 'conversation': 991, 'build': 593, 'technology': 4445, 'fundamentally': 1910, 'communicate': 840, 'join': 2448, 'online': 3056, 'offer': 3032, 'kind': 2485, 'personalized': 3266, 'human': 2161, 'experience': 1659, 'find': 1783, 'visit': 4818, 'local': 2600, 'visual': 4820, 'wide': 4897, 'range': 3608, 'include': 2239, 'campaign': 628, 'website': 4871, 'print': 

# **Train/ Evaluate Model**

In [30]:
# Classifier Algorithm= Support Vector Machine

# Fit the Classifier w the Training Data
# Time the duration of the fit
t1_start = time()

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_Tfidf, Y_train)

t1_stop = time() 
print("Stop/Start Time:", t1_stop, t1_start)   
print(f'Duration of Model in Nanoseconds:',t1_stop-t1_start)

# Use model to get predictions from the Testing data
predictions_SVM = SVM.predict(X_test_Tfidf)

# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score: ", accuracy_score(predictions_SVM, Y_test)*100)

Stop/Start Time: 1586995145.146131 1586995143.7631302
Duration of Model in Nanoseconds: 1.3830008506774902
SVM Accuracy Score:  87.9746835443038


## **Hypertune Model**

In [140]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 100],
              'gamma': [0.001, 0.01, .1, 1],
              'kernel': ['linear','rbf', 'poly', 'sigmoid']}
grid = GridSearchCV(SVM, param_grid, verbose=3)

In [141]:
# Train the model with GridSearch
grid.fit(X_train_Tfidf, Y_train)

Fitting 5 folds for each of 64 candidates, totalling 320 fits
[CV] C=1, gamma=0.001, kernel=linear .................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..... C=1, gamma=0.001, kernel=linear, score=0.853, total=   1.0s
[CV] C=1, gamma=0.001, kernel=linear .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s


[CV] ..... C=1, gamma=0.001, kernel=linear, score=0.874, total=   1.0s
[CV] C=1, gamma=0.001, kernel=linear .................................


...

[CV] ... C=10, gamma=0.001, kernel=sigmoid, score=0.790, total=   1.1s
[CV] C=10, gamma=0.001, kernel=sigmoid ...............................
[CV] ... C=10, gamma=0.001, kernel=sigmoid, score=0.792, total=   1.1s
[CV] C=10, gamma=0.001, kernel=sigmoid ...............................
[CV] ... C=10, gamma=0.001, kernel=sigmoid, score=0.792, total=   1.1s
[CV] C=10, gamma=0.001, kernel=sigmoid ...............................
[CV] ... C=10, gamma=0.001, kernel=sigmoid, score=0.792, total=   1.0s
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV] ..... C=10, gamma=0.01, kernel=linear, score=0.844, total=   1.0s
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV] ..... C=10, gamma=0.01, kernel=linear, score=0.871, total=   1.0s
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV] ..... C=10, gamma=0.01, kernel=linear, score=0.900, total=   1.4s
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV] .

[CV] .......... C=10, gamma=1, kernel=poly, score=0.853, total=   2.0s
[CV] C=10, gamma=1, kernel=sigmoid ...................................
[CV] ....... C=10, gamma=1, kernel=sigmoid, score=0.819, total=   0.7s
[CV] C=10, gamma=1, kernel=sigmoid ...................................
[CV] ....... C=10, gamma=1, kernel=sigmoid, score=0.833, total=   0.8s
[CV] C=10, gamma=1, kernel=sigmoid ...................................
[CV] ....... C=10, gamma=1, kernel=sigmoid, score=0.878, total=   0.8s
[CV] C=10, gamma=1, kernel=sigmoid ...................................
[CV] ....... C=10, gamma=1, kernel=sigmoid, score=0.835, total=   0.8s
[CV] C=10, gamma=1, kernel=sigmoid ...................................
[CV] ....... C=10, gamma=1, kernel=sigmoid, score=0.803, total=   0.8s
[CV] C=100, gamma=0.001, kernel=linear ...............................
[CV] ... C=100, gamma=0.001, kernel=linear, score=0.831, total=   1.0s
[CV] C=100, gamma=0.001, kernel=linear ...............................
[CV] .

[CV] ....... C=100, gamma=0.1, kernel=poly, score=0.792, total=   1.9s
[CV] C=100, gamma=0.1, kernel=poly ...................................
[CV] ....... C=100, gamma=0.1, kernel=poly, score=0.792, total=   1.9s
[CV] C=100, gamma=0.1, kernel=poly ...................................
[CV] ....... C=100, gamma=0.1, kernel=poly, score=0.792, total=   1.9s
[CV] C=100, gamma=0.1, kernel=sigmoid ................................
[CV] .... C=100, gamma=0.1, kernel=sigmoid, score=0.844, total=   1.0s
[CV] C=100, gamma=0.1, kernel=sigmoid ................................
[CV] .... C=100, gamma=0.1, kernel=sigmoid, score=0.871, total=   1.0s
[CV] C=100, gamma=0.1, kernel=sigmoid ................................
[CV] .... C=100, gamma=0.1, kernel=sigmoid, score=0.900, total=   1.4s
[CV] C=100, gamma=0.1, kernel=sigmoid ................................
[CV] .... C=100, gamma=0.1, kernel=sigmoid, score=0.857, total=   1.4s
[CV] C=100, gamma=0.1, kernel=sigmoid ................................
[CV] .

[Parallel(n_jobs=1)]: Done 320 out of 320 | elapsed:  6.2min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto', kernel='linear', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 5, 10, 100], 'gamma': [0.001, 0.01, 0.1, 1],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [142]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 5, 'gamma': 1, 'kernel': 'rbf'}
0.8856388466134847


In [143]:
# Train the hypertuned model
SVM2 = svm.SVC(kernel='rbf', C= 5, gamma= 1)
SVM2.fit(X_train_Tfidf, Y_train)

SVC(C=5, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [144]:
# Hypertuned Model Accuracy
print('SVM2 Acc Score: %.3f' % SVM2.score(X_test_Tfidf, Y_test))

SVM2 Acc Score: 0.903


In [145]:
from sklearn.metrics import classification_report
predictions = SVM2.predict(X_test_Tfidf)
print("SVM2 Classification Report: \n" + classification_report(Y_test, predictions, target_names=['real','fake']))

SVM2 Classification Report: 
              precision    recall  f1-score   support

        real       0.90      0.99      0.94       777
        fake       0.92      0.51      0.65       171

    accuracy                           0.90       948
   macro avg       0.91      0.75      0.80       948
weighted avg       0.90      0.90      0.89       948



## **Save Model**

In [146]:
# import joblib
# joblib.dump(SVM2, 'nlp_svm2.h5')

['nlp_svm2.h5']